# 22. lab 44 - obliczenia


## Imports definitions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os


# Konfiguracja ścieżek
output_dir = "22-lab-44/report/img"
results_file = "22-lab-44/report/wyniki_obliczen.txt"
os.makedirs(output_dir, exist_ok=True)

# Funkcja pomocnicza do wypisywania i zapisywania pojedynczych wyników
def log_result(text, filepath=results_file):
    print(text)
    with open(filepath, "a", encoding="utf-8") as f:
        f.write(text + "\n")

# Funkcja pomocnicza do wypisywania i zapisywania całych tabel (lista linii)
def log_table(lines, filepath=results_file):
    for line in lines:
        print(line)
    with open(filepath, "a", encoding="utf-8") as f:
        for line in lines:
            f.write(line + "\n")
    print("\n") # odstęp dla czytelności

# Wyczyszczenie pliku z wynikami na początku
with open(results_file, "w", encoding="utf-8") as f:
    f.write("--- WYNIKI OBLICZEŃ LAB 44 ---\n\n")

# --- DANE POMIAROWE I STAŁE ---

# Częstotliwość sieci
f_net = 50.0 # Hz
omega = 2 * np.pi * f_net

# Rezystancje drutu (dane z instrukcji)
R_ab = 36.0 # Ohm
R_bc = 16.0 # Ohm
R_cd = 33.0 # Ohm

# 1. Cewki (L)
# Prądy w mA (zostaną przeliczone na A)
I_L1_raw = np.array([10.0, 20.0, 29.9, 39.7, 49.7, 59.5, 69.1, 79.3, 89.3, 99.0])
U_L1 = np.array([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00])

I_L1L2_raw = np.array([3.7, 7.5, 11.2, 14.9, 18.8, 22.4, 26.1, 29.9, 33.7, 37.4])
U_L1L2 = np.array([0.50, 1.00, 1.50, 2.00, 2.51, 3.00, 3.50, 3.99, 4.50, 5.00])

I_L1L2L3_raw = np.array([1.5, 3.0, 4.7, 6.3, 7.9, 9.4, 11.1, 12.7, 14.3, 16.0])
U_L1L2L3 = np.array([0.48, 0.96, 1.50, 1.98, 2.50, 2.97, 3.48, 3.98, 4.49, 5.02])

# Konwersja prądu na Ampery
I_L1 = I_L1_raw * 1e-3
I_L1L2 = I_L1L2_raw * 1e-3
I_L1L2L3 = I_L1L2L3_raw * 1e-3

# 2. Kondensatory (C)
# C1 = C
U_C1 = np.array([0.48, 0.96, 1.49, 1.98, 2.49, 2.99, 3.51, 4.01, 4.53, 5.02])
I_C1_raw = np.array([0.48, 1.00, 1.60, 2.10, 2.70, 3.20, 3.80, 4.40, 5.00, 5.50])

# C2 = 2C
U_C2 = np.array([0.50, 1.02, 1.50, 1.98, 2.47, 3.05, 3.52, 3.98, 4.52, 5.03])
I_C2_raw = np.array([1.0, 2.2, 3.3, 4.3, 5.5, 6.8, 7.8, 8.9, 10.1, 11.3])

# C3 = 4C
U_C3 = np.array([0.48, 0.99, 1.52, 2.00, 2.53, 3.05, 3.48, 4.01, 4.51, 5.02])
I_C3_raw = np.array([2.4, 5.0, 7.7, 10.1, 12.8, 15.5, 17.7, 20.4, 22.4, 25.5])

# Konwersja prądu na Ampery
I_C1 = I_C1_raw * 1e-3
I_C2 = I_C2_raw * 1e-3
I_C3 = I_C3_raw * 1e-3

log_result("Dane zaimportowane pomyślnie.")
log_result(f"Przyjęto częstość kołową omega = {omega:.2f} rad/s (dla f=50Hz)")
log_result(f"Przyjęto rezystancje: Rab={R_ab}, Rbc={R_bc}, Rcd={R_cd} Ohm\n")

## --- task 1: coils analysis ---

In [ ]:
# --- ZADANIE 1: ANALIZA CEWEK (L) ---

# Rezystancje z instrukcji (suma dla kolejnych układów)
R_L1 = R_ab
R_L1L2 = R_ab + R_bc
R_L1L2L3 = R_ab + R_bc + R_cd

# Lista danych pomiarowych (Układy narastające)
# Kolejność jest WAŻNA: L1, potem L1+L2, potem L1+L2+L3
coils_data = [
    ("L1 (a-b)",       I_L1,     U_L1,     R_L1),
    ("L1+L2 (a-c)",    I_L1L2,   U_L1L2,   R_L1L2),
    ("L1+L2+L3 (a-d)", I_L1L2L3, U_L1L2L3, R_L1L2L3)
]

results_systems = [] # Tu trzymamy wyniki dla całych układów (a-b, a-c...)

for name, I, U, R in coils_data:
    # 1. Regresja liniowa U = a*I + b
    res = linregress(I, U)
    a = res.slope       # = |Z|
    u_a = res.stderr    # = u(|Z|)
    b = res.intercept
    r2 = res.rvalue**2

    # 2. Obliczenie indukcyjności CAŁEGO UKŁADU
    if a > R:
        L_sys = np.sqrt(a**2 - R**2) / omega
        # Niepewność metodą różniczki zupełnej
        dL_da = a / (omega**2 * L_sys)
        u_L_sys = dL_da * u_a
    else:
        L_sys = 0.0
        u_L_sys = 0.0
        
    results_systems.append({
        "name": name, 
        "L_sys": L_sys, 
        "u_L_sys": u_L_sys,
        "a": a, "u_a": u_a, "b": b, "R2": r2, "R_wire": R
    })

# --- OBLICZANIE INDUKCYJNOŚCI POSZCZEGÓLNYCH CEWEK (L1, L2, L3) ---
# Zakładamy addytywność pomiarów (L_2 = L_{1+2} - L_1)

L_val = [] # Lista wartości [L1, L2, L3]
u_L_val = [] # Lista niepewności

# Cewka L1 to po prostu wynik pierwszego pomiaru
l1 = results_systems[0]['L_sys']
u_l1 = results_systems[0]['u_L_sys']
L_val.append(l1)
u_L_val.append(u_l1)

# Cewka L2 = L(a-c) - L(a-b)
l2 = results_systems[1]['L_sys'] - results_systems[0]['L_sys']
u_l2 = np.sqrt(results_systems[1]['u_L_sys']**2 + results_systems[0]['u_L_sys']**2)
L_val.append(l2)
u_L_val.append(u_l2)

# Cewka L3 = L(a-d) - L(a-c)
l3 = results_systems[2]['L_sys'] - results_systems[1]['L_sys']
u_l3 = np.sqrt(results_systems[2]['u_L_sys']**2 + results_systems[1]['u_L_sys']**2)
L_val.append(l3)
u_L_val.append(u_l3)

# Zapisujemy wyniki cewek do listy słowników dla łatwego drukowania
results_parts = [
    {"part": "Cewka L1", "L": L_val[0], "u_L": u_L_val[0]},
    {"part": "Cewka L2", "L": L_val[1], "u_L": u_L_val[1]},
    {"part": "Cewka L3", "L": L_val[2], "u_L": u_L_val[2]},
]

# --- GENEROWANIE TABEL TEKSTOWYCH ---

# Tabela 1: Pomiary układów (dla sprawdzenia liniowości)
lines = []
lines.append("TABELA 1: Wyniki regresji dla badanych układów (całościowych)")
lines.append(f"{'Układ':<20} | {'|Z| [Ohm]':<12} | {'L_ukladu [H]':<15} | {'R^2':<8}")
lines.append("-" * 65)
for r in results_systems:
    lines.append(f"{r['name']:<20} | {r['a']:<12.2f} | {r['L_sys']:<15.4f} | {r['R2']:<8.4f}")
lines.append("")
log_table(lines)

# Tabela 2: WYNIKI KOŃCOWE DLA CEWEK
lines = []
lines.append("TABELA 2: Obliczone współczynniki samoindukcji poszczególnych cewek")
lines.append("(Obliczone jako różnice indukcyjności kolejnych układów)")
lines.append("-" * 60)
lines.append(f"{'Cewka':<20} | {'L [H]':<12} | {'u(L) [H]':<12}")
lines.append("-" * 60)
for r in results_parts:
    lines.append(f"{r['part']:<20} | {r['L']:<12.5f} | {r['u_L']:<12.5f}")
lines.append("-" * 60)
lines.append(f"Całkowita L (suma):  {sum(L_val):.5f} H")
lines.append(f"L zmierzona (a-d):   {results_systems[2]['L_sys']:.5f} H")
lines.append("")
log_table(lines)

# --- WYKRES ---
plt.figure(figsize=(9, 6))

# Punkty pomiarowe (przeliczamy I z powrotem na mA dla czytelności wykresu)
plt.plot(U_L1, I_L1*1000, 'bo', label='L1 (a-b)')
plt.plot(U_L1L2, I_L1L2*1000, 'gs', label='L1+L2 (a-c)')
plt.plot(U_L1L2L3, I_L1L2L3*1000, 'r^', label='L1+L2+L3 (a-d)')

# Proste regresji
vals_U = np.linspace(0, 5.2, 10)
# Iterujemy po wynikach systemów (results_systems)
for r, color in zip(results_systems, ['b', 'g', 'r']):
    # Odwracamy równanie regresji U = a*I + b  =>  I = (U - b)/a
    # Mnożymy *1000 aby uzyskać mA
    plt.plot(vals_U, ((vals_U - r['b']) / r['a']) * 1000, color+'--', alpha=0.5)

plt.xlabel('Napięcie U [V]')
plt.ylabel('Natężenie prądu I [mA]')
plt.title('Charakterystyki prądowo-napięciowe układów cewek')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

save_path = os.path.join(output_dir, "charakterystyka_cewek.png")
plt.savefig(save_path, dpi=300)
print(f"Wykres zapisano: {save_path}")
plt.show()

## --- task 2: capacitors analysis ---

In [ ]:
log_result("--- ZADANIE 2: KONDENSATORY ---")

def regresja(U, I):
    wynik = linregress(U, I)
    return wynik.slope, wynik.intercept

# Prawo Ohma dla kondensatora: I = U / Xc = U * (wC)
# Zatem w równaniu prostej I = a * U, współczynnik a = wC
# Stąd pojemność C = a / omega

a1, b1 = regresja(U_C1, I_C1)
a2, b2 = regresja(U_C2, I_C2)
a3, b3 = regresja(U_C3, I_C3)

# Reaktancja pojemnościowa Xc = 1/a = U/I
Xc1 = 1 / a1
Xc2 = 1 / a2
Xc3 = 1 / a3

# Obliczenie pojemności C = a / omega
C_val1 = a1 / omega
C_val2 = a2 / omega
C_val3 = a3 / omega

log_result("Wyniki regresji (współczynniki kierunkowe 'a'), reaktancje Xc i pojemności C:")
log_result(f"C1: a = {a1:.4e} A/V | Xc = {Xc1:.2f} Ω | C = {C_val1*1e6:.2f} µF")
log_result(f"C2: a = {a2:.4e} A/V | Xc = {Xc2:.2f} Ω | C = {C_val2*1e6:.2f} µF")
log_result(f"C3: a = {a3:.4e} A/V | Xc = {Xc3:.2f} Ω | C = {C_val3*1e6:.2f} µF")

# Weryfikacja stosunków pojemności (C2 powinno być ~2*C1, C3 ~4*C1)
log_result(f"Stosunek C2/C1 = {C_val2/C_val1:.2f} (oczekiwane ~2.0)")
log_result(f"Stosunek C3/C1 = {C_val3/C_val1:.2f} (oczekiwane ~4.0)")

plt.figure(figsize=(8, 6))
plt.plot(U_C1, I_C1, marker='o', linestyle="None", label="C1")
plt.plot(U_C2, I_C2, marker='s', linestyle="None", label="C2")
plt.plot(U_C3, I_C3, marker='^', linestyle="None", label="C3")

# Dorysowanie prostych regresji
U_plot = np.linspace(0, 5.2, 100)
plt.plot(U_plot, a1*U_plot + b1, 'b--', alpha=0.5, label=f'Fit C1')
plt.plot(U_plot, a2*U_plot + b2, 'orange', linestyle='--', alpha=0.5, label=f'Fit C2')
plt.plot(U_plot, a3*U_plot + b3, 'g--', alpha=0.5, label=f'Fit C3')

plt.xlabel("Napięcie U [V]")
plt.ylabel("Prąd I [A]")
plt.title(f"Charakterystyki I=f(U) dla kondensatorów")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

save_path = os.path.join(output_dir, "charakterystyka_kondensatorow.png")
plt.savefig(save_path, dpi=300)
print(f"Wykres zapisano: {save_path}")
plt.show()